In [2]:
# Import Dependencies
import pymongo
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
from splinter import Browser
from urllib.parse import urljoin
from pymongo import MongoClient
from sqlalchemy import create_engine



In [3]:
# Set path to browser & start splinter
browser = Browser('chrome')
url = 'http://quotes.toscrape.com/'


In [4]:
html = browser.html
soup_obj = soup(html, 'html.parser')

In [5]:
# Create multiple functions to scrape data
def get_quote(soup_obj):
    data = {}
    data['text'] = soup_obj.find("span", class_ = "text").text
    data['author'] = soup_obj.find("small", class_ = "author").text
    link = soup_obj.a['href']
    author_url = urljoin(url, link)
    data['author'] = get_author_info(author_url)
    data['tags'] = get_tags(soup_obj)
    return data

In [6]:
def get_author_info(url):
    author_info = {}
    response = requests.get(url)
    soup1 =  soup(response.text, 'lxml') 
    author_info['name'] = soup1.find('h3', class_="author-title").text.strip()   
    author_info['born'] = soup1.find('span', class_="author-born-date").text.strip()
    author_info['location'] = soup1.find('span', class_="author-born-location").text.strip()
    author_info['description'] = soup1.find('div', class_="author-description").text.strip()
    return author_info

In [7]:
def get_tags(soup_obj):
    tags_list = []
    for tag in soup_obj.findAll("a", class_ = "tag"):
        tags_list.append(tag.text) 
    return tags_list

In [8]:
def all_quotes(page_number):
    quote_list = []
    html = browser.html
    soup_obj = soup(html, 'html.parser')
    quote_block = soup_obj.findAll("div", class_="quote")
    num_quote = 10
    quote_id = (page_number -1)  * num_quote 
    for block in quote_block:
        quote_id = quote_id +1
        quote = get_quote(block)
        quote['id'] = quote_id
        quote_list.append(quote) 
    return quote_list

In [9]:
def scrape_all(url):

    page_number = 0
    final_quotes = []

    for x in range(1,11):
        page_number = page_number +1
        browser.visit(f'http://quotes.toscrape.com/page/{x}/')
        html = browser.html
        soup_obj = soup(html, 'html.parser')

        current_quotes = all_quotes(page_number)
        final_quotes = final_quotes + current_quotes
    return final_quotes
        

In [10]:
# Function to save data into mongodb
def save(final_quotes):
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client.quotes_db
    collection = db.quotes
    collection.insert_many(final_quotes)

In [11]:
final_quotes = scrape_all(url)
browser.quit()

In [58]:
save(final_quotes)

In [12]:
# Reads mongodb data
def extract_mongo():
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client.quotes_db
    collection = db.quotes
    return collection

In [13]:
# Creates tables to allow mongodb tansfer to postgres
def create_tables(extract):
    quotes_table = []
    authors = []
    authors_table = []
    tags_table = []
    for x in extract.find({}):
        quote = {}
        quote['id'] = x['id']
        quote['author_name'] = x['author']['name']
        quote['text'] = x['text']
        quotes_table.append(quote)

        author = {}
        author['name'] = x['author']['name']
        author['born'] =  x['author']['born']
        author['location'] =  x['author']['location']
        author['description'] = x['author']['description']
        if (author['name'] not in authors):
            authors_table.append(author)
            authors.append(author['name'])

        for tag in x['tags']:
            tags_table.append({'quote_id':x['id'] , 'tag' : tag})    
            
    return (quotes_table , authors_table ,tags_table)

In [14]:
extract = extract_mongo()

In [15]:
(quotes, authors, tags) = create_tables(extract)
quotes_df = pd.DataFrame(quotes)
authors_df = pd.DataFrame(authors)
tags_df = pd.DataFrame(tags)

In [16]:
authors_df.head()

,name,born,location,description
0,Albert Einstein,"March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
1,J.K. Rowling,"July 31, 1965","in Yate, South Gloucestershire, England, The U...",See also: Robert GalbraithAlthough she writes ...
2,Jane Austen,"December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",Jane Austen was an English novelist whose work...
3,Marilyn Monroe,"June 01, 1926",in The United States,Marilyn Monroe (born Norma Jeane Mortenson; Ju...
4,André Gide,"November 22, 1869","in Paris, France",André Paul Guillaume Gide was a French author ...


In [17]:
# Postgres connection string (quotes_db database)
connection_string = f"postgres:postgres@localhost:5432/quotes_db"
engine = create_engine(f'postgresql://{connection_string}')

In [18]:
quotes_table_query = '''
    create table quotes(     id INTEGER PRIMARY KEY,    
    author_name varchar(32),    
    text varchar(1500))
'''

tags_table_query = '''
    create table tags(    quote_id INTEGER,    
    tag varchar(32))
'''

author_table_query = '''
    create table author( name varchar(32) PRIMARY KEY,    
    born varchar(32),
    location varchar(1000),
    description varchar(10000))
'''

In [19]:
# First attempt to create tables in postgres

#  tables = {'quotes' : quotes_table_query.strip(), 
#           'tags' : tags_table_query.strip(),
#           'author' : author_table_query.strip()
#          }

In [109]:
# for table in tables.keys():
#     engine.execute(f'drop table IF EXISTS {table}')

In [21]:
# for query in tables.items():
#     engine.execute(f'{query}')

In [20]:
# Code below brings data from mongodb to postgres
engine.table_names() 

['quotes', 'tags', 'author']

In [22]:
quotes_df.to_sql(name='quotes', con=engine, if_exists='append', index=False) 

In [23]:
tags_df.to_sql(name='tags', con=engine, if_exists='append', index=False) 

In [26]:
authors_df.to_sql(name='author', con=engine, if_exists='append', index=False) 